# visualizer

Visualization module for creating interactive maps of poaching detection results.

In [ ]:
import foliumimport pandas as pdimport numpy as npfrom datetime import datetimeimport osfrom folium import pluginsfrom utils import save_dataframe

## Code

In [ ]:
Visualization module for creating interactive maps of poaching detection results.class MapVisualizer:    def __init__(self, center_lat=-1.2921, center_lon=36.8219, zoom_start=12):        Initialize the map visualizer.        Args:            center_lat: Center latitude for the map            center_lon: Center longitude for the map            zoom_start: Initial zoom level        self.center_lat = center_lat        self.center_lon = center_lon        self.zoom_start = zoom_start    def create_base_map(self):        print("Creating base map...")        # Create base map        m = folium.Map(            location=[self.center_lat, self.center_lon],            zoom_start=self.zoom_start,            tiles='OpenStreetMap'        )        # Add satellite tile layer        folium.TileLayer(            tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',            attr='Esri',            name='Satellite',            overlay=False,            control=True        ).add_to(m)        return m    def add_gps_tracks(self, map_obj, gps_df, anomaly_df=None):        Add GPS tracks to the map.        Args:            map_obj: Folium map object            gps_df: DataFrame with GPS tracking data            anomaly_df: DataFrame with anomaly detection results        print("Adding GPS tracks to map...")        if gps_df.empty:            return        # Group by animal        for animal_id in gps_df['animal_id'].unique():            animal_data = gps_df[gps_df['animal_id'] == animal_id].copy()            animal_data = animal_data.sort_values('timestamp')            # Create track coordinates            track_coords = list(zip(animal_data['latitude'], animal_data['longitude']))            # Determine track color based on anomalies            if anomaly_df is not None and not anomaly_df.empty:                animal_anomalies = anomaly_df[anomaly_df['animal_id'] == animal_id]                has_anomalies = len(animal_anomalies[animal_anomalies['is_anomaly']]) > 0                color = 'red' if has_anomalies else 'blue'            else:                color = 'blue'            # Add track line            folium.PolyLine(                track_coords,                color=color,                weight=3,                opacity=0.7,                popup=f"Animal Track: {animal_id}"            ).add_to(map_obj)            # Add start and end markers            if len(track_coords) > 0:                # Start marker                folium.CircleMarker(                    track_coords[0],                    radius=8,                    color='green',                    fill=True,                    popup=f"Start: {animal_id}"                ).add_to(map_obj)                # End marker                folium.CircleMarker(                    track_coords[-1],                    radius=8,                    color='orange',                    fill=True,                    popup=f"End: {animal_id}"                ).add_to(map_obj)    def add_anomaly_points(self, map_obj, anomaly_df):        Add anomaly points to the map.        Args:            map_obj: Folium map object            anomaly_df: DataFrame with anomaly detection results        print("Adding anomaly points to map...")        if anomaly_df.empty:            return        # Filter for anomalies only        anomalies = anomaly_df[anomaly_df['is_anomaly']].copy()        if anomalies.empty:            return        # Create anomaly feature group        anomaly_group = folium.FeatureGroup(name="GPS Anomalies")        for _, anomaly in anomalies.iterrows():            # Determine marker color based on severity            if anomaly['anomaly_severity'] == 'High':                color = 'red'                size = 12            elif anomaly['anomaly_severity'] == 'Medium':                color = 'orange'                size = 10            else:                color = 'yellow'                size = 8            # Create popup text            popup_text = f"""            <b>GPS Anomaly</b><br>            Animal: {anomaly['animal_id']}<br>            Time: {anomaly['timestamp']}<br>            Severity: {anomaly['anomaly_severity']}<br>            Score: {anomaly['anomaly_score']:.3f}            folium.CircleMarker(                [anomaly['latitude'], anomaly['longitude']],                radius=size,                color=color,                fill=True,                popup=folium.Popup(popup_text, max_width=200)            ).add_to(anomaly_group)        anomaly_group.add_to(map_obj)    def add_detection_points(self, map_obj, detections_df):        Add human/vehicle detection points to the map.        Args:            map_obj: Folium map object            detections_df: DataFrame with image detections        print("Adding detection points to map...")        if detections_df.empty:            return        # Create detection feature group        detection_group = folium.FeatureGroup(name="Object Detections")        # Define icons for different detection types        icon_map = {            'person': 'user',            'car': 'car',            'truck': 'truck',            'motorcycle': 'motorcycle',            'bus': 'bus'        }        for _, detection in detections_df.iterrows():            # Determine icon and color            detection_class = detection['class']            if detection_class == 'person':                icon = 'user'                color = 'red'            else:                icon = 'car'                color = 'blue'            # Create popup text            popup_text = f"""            <b>Object Detection</b><br>            Type: {detection_class}<br>            Confidence: {detection['confidence']:.3f}<br>            Image: {detection['image_id']}<br>            Time: {detection['timestamp']}            folium.Marker(                [detection['detection_lat'], detection['detection_lon']],                icon=folium.Icon(icon=icon, color=color),                popup=folium.Popup(popup_text, max_width=200)            ).add_to(detection_group)        detection_group.add_to(map_obj)    def add_alert_zones(self, map_obj, alerts_df):        Add alert zones to the map.        Args:            map_obj: Folium map object            alerts_df: DataFrame with generated alerts        print("Adding alert zones to map...")        if alerts_df.empty:            return        # Create alert feature group        alert_group = folium.FeatureGroup(name="Poaching Alerts")        for _, alert in alerts_df.iterrows():            # Determine circle color and size based on alert level            if alert['alert_level'] == 'High':                color = 'red'                fill_color = 'red'                radius = 200            elif alert['alert_level'] == 'Medium':                color = 'orange'                fill_color = 'orange'                radius = 150            else:                color = 'yellow'                fill_color = 'yellow'                radius = 100            # Create popup text            popup_text = f"""            <b>Poaching Alert</b><br>            Type: {alert['alert_type']}<br>            Level: {alert['alert_level']}<br>            Time: {alert['timestamp']}<br>            Description: {alert['description']}            folium.Circle(                [alert['latitude'], alert['longitude']],                radius=radius,                color=color,                fill=True,                fill_color=fill_color,                fill_opacity=0.3,                popup=folium.Popup(popup_text, max_width=300)            ).add_to(alert_group)        alert_group.add_to(map_obj)    def add_heatmap(self, map_obj, data_df, lat_col='latitude', lon_col='longitude', weight_col=None):        Add heatmap layer to the map.        Args:            map_obj: Folium map object            data_df: DataFrame with coordinate data            lat_col: Latitude column name            lon_col: Longitude column name            weight_col: Weight column for heatmap intensity        print("Adding heatmap layer...")        if data_df.empty:            return        # Prepare heatmap data        heatmap_data = []        for _, row in data_df.iterrows():            lat = row[lat_col]            lon = row[lon_col]            weight = row[weight_col] if weight_col and weight_col in row else 1            heatmap_data.append([lat, lon, weight])        # Add heatmap        plugins.HeatMap(            heatmap_data,            name="Detection Heatmap",            show=True,            overlay=True,            control=True        ).add_to(map_obj)    def create_poaching_map(self, gps_df, anomaly_df, detections_df, alerts_df, output_path='output/poaching_alerts_map.html'):        Create the complete poaching detection map.        Args:            gps_df: DataFrame with GPS tracking data            anomaly_df: DataFrame with anomaly detection results            detections_df: DataFrame with image detections            alerts_df: DataFrame with generated alerts            output_path: Path to save the map HTML file        print("Creating comprehensive poaching detection map...")        # Create base map        m = self.create_base_map()        # Add all layers        self.add_gps_tracks(m, gps_df, anomaly_df)        self.add_anomaly_points(m, anomaly_df)        self.add_detection_points(m, detections_df)        self.add_alert_zones(m, alerts_df)        # Add heatmap for detections        if not detections_df.empty:            self.add_heatmap(m, detections_df, 'detection_lat', 'detection_lon', 'confidence')        # Add layer control        folium.LayerControl().add_to(m)        # Add title        title_html = '''        <h3 align="center" style="font-size:20px"><b>Illegal Poaching Detection Map</b></h3>        m.get_root().html.add_child(folium.Element(title_html))        # Save map        os.makedirs(os.path.dirname(output_path), exist_ok=True)        m.save(output_path)        print(f"Map saved to {output_path}")        return m    def create_summary_dashboard(self, gps_df, anomaly_df, detections_df, alerts_df):        Create a summary dashboard with statistics.        Args:            gps_df: DataFrame with GPS tracking data            anomaly_df: DataFrame with anomaly detection results            detections_df: DataFrame with image detections            alerts_df: DataFrame with generated alerts        Returns:            Dictionary with summary statistics        print("Creating summary dashboard...")        # Calculate statistics        total_gps_points = len(gps_df)        total_anomalies = anomaly_df['is_anomaly'].sum() if not anomaly_df.empty else 0        total_detections = len(detections_df)        total_alerts = len(alerts_df)        # Detection breakdown        detection_breakdown = detections_df['class'].value_counts().to_dict() if not detections_df.empty else {}        # Alert breakdown        alert_breakdown = alerts_df['alert_level'].value_counts().to_dict() if not alerts_df.empty else {}        # Animal statistics        unique_animals = gps_df['animal_id'].nunique() if not gps_df.empty else 0        animals_with_anomalies = anomaly_df[anomaly_df['is_anomaly']]['animal_id'].nunique() if not anomaly_df.empty else 0        summary = {            'total_gps_points': total_gps_points,            'total_anomalies': total_anomalies,            'total_detections': total_detections,            'total_alerts': total_alerts,            'unique_animals': unique_animals,            'animals_with_anomalies': animals_with_anomalies,            'detection_breakdown': detection_breakdown,            'alert_breakdown': alert_breakdown,            'anomaly_rate': (total_anomalies / total_gps_points * 100) if total_gps_points > 0 else 0        }        return summary

## Test Code

In [ ]:
    # Test the visualizer    visualizer = MapVisualizer()    # Load sample data    try:        gps_df = pd.read_csv('output/gps_tracking_data.csv')        gps_df['timestamp'] = pd.to_datetime(gps_df['timestamp'])        anomaly_df = pd.read_csv('output/gps_anomalies.csv')        anomaly_df['timestamp'] = pd.to_datetime(anomaly_df['timestamp'])        detections_df = pd.read_csv('output/image_detections.csv')        detections_df['timestamp'] = pd.to_datetime(detections_df['timestamp'])        alerts_df = pd.read_csv('output/poaching_alerts.csv')        alerts_df['timestamp'] = pd.to_datetime(alerts_df['timestamp'])        # Create map        map_obj = visualizer.create_poaching_map(gps_df, anomaly_df, detections_df, alerts_df)        # Create summary        summary = visualizer.create_summary_dashboard(gps_df, anomaly_df, detections_df, alerts_df)        print("Summary Dashboard:", summary)    except FileNotFoundError as e:        print(f"Data files not found: {e}")        print("Run the main pipeline first to generate data.")